In [4]:
import statsmodels.api as sm
import pandas as pd

In [15]:
price0 = pd.read_csv('R1_day0.csv',header=0,delimiter=';')
price1 = pd.read_csv('R1_day-1.csv',header=0,delimiter=';')
price2 = pd.read_csv('R1_day-2.csv',header=0,delimiter=';')

In [27]:
price0_SF = price0[price0['product']=='STARFRUIT']['mid_price']
price1_SF = price1[price1['product']=='STARFRUIT']['mid_price']
price2_SF = price2[price2['product']=='STARFRUIT']['mid_price']

In [32]:
ts = pd.concat([price2_SF,price1_SF,price0_SF]).reset_index(drop=True)

# Model v1

In [61]:
ts

0        5000.0
1        5002.0
2        5000.0
3        4999.5
4        4998.5
          ...  
29995    5052.5
29996    5051.0
29997    5053.5
29998    5049.5
29999    5051.0
Name: mid_price, Length: 30000, dtype: float64

In [60]:
LR_df

,lag1,lag2,lag3,lag4,target
0,4999.5,5000.0,5002.0,5000.0,4998.5
1,4998.5,4999.5,5000.0,5002.0,5002.0
2,5002.0,4998.5,4999.5,5000.0,4999.5
3,4999.5,5002.0,4998.5,4999.5,5000.0
4,5000.0,4999.5,5002.0,4998.5,5002.5
...,...,...,...,...,...
29991,5051.5,5052.0,5051.5,5052.0,5052.5
29992,5052.5,5051.5,5052.0,5051.5,5051.0
29993,5051.0,5052.5,5051.5,5052.0,5053.5
29994,5053.5,5051.0,5052.5,5051.5,5049.5


In [41]:
LR_df = pd.DataFrame({'lag1': ts.shift(1),
              'lag2': ts.shift(2),
              'lag3': ts.shift(3),
              'lag4': ts.shift(4),
              'target': ts})

In [46]:
LR_df = LR_df.dropna().reset_index(drop=True)

In [51]:
X = LR_df[['lag1','lag2','lag3','lag4']]
y = LR_df['target']
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

In [54]:
results.params

const    2.356494
lag1     0.341769
lag2     0.261069
lag3     0.207707
lag4     0.188988
dtype: float64

In [55]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 3.939e+06
Date:                Wed, 10 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:36:39   Log-Likelihood:                -52824.
No. Observations:               29996   AIC:                         1.057e+05
Df Residuals:                   29991   BIC:                         1.057e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3565      1.268      1.858      0.063      -0.130       4.843
lag1           0.3418      0.006     60.274      0.000       0.331       0.353
lag2           0.2611      0.006     44.376      0.000       0.250       0.273
lag3           0.2077      0.006     35.304      0.000       0.196       0.219
lag4           0.1890      0.006     33.328      0.000       0.178       0.200
==============================================================================
Omnibus:                      437.009   Durbin-Watson:                   2.053
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              831.420
Skew:                           0.031   Prob(JB):                    2.88e-181
Kurtosis:                       3.813   Cond. No.                     1.57e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.57e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Model 2

In [56]:
LR_df_v2 = pd.DataFrame({'lag1': ts.shift(1),
              'lag5': ts.shift(5),
              'target': ts})

In [57]:
LR_df_v2 = LR_df_v2.dropna().reset_index(drop=True)

In [58]:
X = LR_df_v2[['lag1','lag5']]
y = LR_df_v2['target']
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

In [59]:
results.params

const    3.881266
lag1     0.591584
lag5     0.407646
dtype: float64

# Model 3

In [62]:
ts.rolling(4).std()

0             NaN
1             NaN
2             NaN
3        1.108678
4        1.471960
           ...   
29995    0.478714
29996    0.645497
29997    1.108678
29998    1.750000
29999    1.658312
Name: mid_price, Length: 30000, dtype: float64

In [64]:
LR_df_3 = pd.DataFrame({'lag1': ts.shift(1),
              'lag2': ts.shift(2),
              'lag3': ts.shift(3),
              'lag4': ts.shift(4),
              'rolling_std': ts.rolling(4).std(),
              'target': ts})


In [66]:
LR_df_3 = LR_df_3.dropna()

In [68]:
X = LR_df_3[['lag1','lag2','lag3','lag4','rolling_std']]
y = LR_df_3['target']
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

# Model 4

In [80]:
from sklearn.linear_model import Ridge

LR_df = pd.DataFrame({'lag1': ts.shift(1),
              'lag2': ts.shift(2),
              'lag3': ts.shift(3),
              'lag4': ts.shift(4),
              'target': ts})

LR_df = LR_df.dropna().reset_index(drop=True)

X = LR_df[['lag1','lag2','lag3','lag4']]
y = LR_df['target']

ridge_model = Ridge(alpha=0.1)

ridge_model.fit(X, y)

coefficients = ridge_model.coef_
intercept = ridge_model.intercept_


In [81]:
coefficients

array([0.34176848, 0.26106907, 0.20770686, 0.18898855])

In [82]:
intercept

2.356498442803968

# Model 5

In [103]:
from sklearn.linear_model import Lasso

LR_df = pd.DataFrame({'lag1': ts.shift(1),
              'lag2': ts.shift(2),
              'lag3': ts.shift(3),
              'lag4': ts.shift(4),
              'target': ts})

LR_df = LR_df.dropna().reset_index(drop=True)

X = LR_df[['lag1','lag2','lag3','lag4']]
y = LR_df['target']

lasso_model = Lasso(alpha=0.5)

lasso_model.fit(X, y)

coefficients = lasso_model.coef_
intercept = lasso_model.intercept_

/Users/jadonng/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.925e+03, tolerance: 3.130e+03
  model = cd_fast.enet_coordinate_descent(


In [104]:
coefficients

array([0.36342121, 0.34247111, 0.20332592, 0.08998536])

In [105]:
intercept

4.015274972830412

In [106]:
mean_squared_error(lasso_model.predict(X),y)

2.0086874249322846

# Model v6

In [97]:
pd.DataFrame({'lag1': ts.shift(1),
              'lag2': ts.shift(2),
              'lag3': ts.shift(3),
              'lag4': ts.shift(4),
              'target': ts.shift(-5)})

,lag1,lag2,lag3,lag4,target
0,NaN,NaN,NaN,NaN,5002.0
1,5000.0,NaN,NaN,NaN,4999.5
2,5002.0,5000.0,NaN,NaN,5000.0
3,5000.0,5002.0,5000.0,NaN,5002.5
4,4999.5,5000.0,5002.0,5000.0,5002.5
...,...,...,...,...,...
29995,5051.5,5052.0,5051.5,5052.0,NaN
29996,5052.5,5051.5,5052.0,5051.5,NaN
29997,5051.0,5052.5,5051.5,5052.0,NaN
29998,5053.5,5051.0,5052.5,5051.5,NaN


In [117]:
from sklearn.linear_model import Lasso

LR_df_5 = pd.DataFrame({'lag1': ts.shift(1),
              'lag2': ts.shift(2),
              'lag3': ts.shift(3),
              'lag4': ts.shift(4),
              'target': ts.shift(-5)})

LR_df_5 = LR_df_5.dropna().reset_index(drop=True)

X = LR_df_5[['lag1','lag2','lag3','lag4']]
y = LR_df_5['target']

lasso_model = Lasso(alpha=0.5)

lasso_model.fit(X, y)

coefficients = lasso_model.coef_
intercept = lasso_model.intercept_

/Users/jadonng/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.796e+03, tolerance: 3.129e+03
  model = cd_fast.enet_coordinate_descent(


In [118]:
coefficients

array([0.37049416, 0.34303531, 0.19067047, 0.09452187])

In [119]:
intercept

6.4507169901571615

In [120]:
lasso_model.predict(X)

array([5000.25589209, 4999.52158307, 5000.19089836, ..., 5052.66882127,
       5052.09327794, 5051.91633689])

In [121]:
from sklearn.metrics import mean_squared_error
mean_squared_error(lasso_model.predict(X),y)

2.8097979369204427

# Analysis

In [124]:
day0_ame = price0[price0['product']=='AMETHYSTS']

In [126]:
day0_ame.query('bid_price_1 > 10000')['bid_price_1']

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
10,0,500,AMETHYSTS,10002,1,9996.0,2.0,9995.0,22.0,10004,2,10005.0,22.0,NaN,NaN,10003.0,0.0
31,0,1500,AMETHYSTS,10002,1,9996.0,2.0,9995.0,20.0,10004,2,10005.0,20.0,NaN,NaN,10003.0,0.0
58,0,2900,AMETHYSTS,10002,10,9996.0,2.0,9995.0,23.0,10004,2,10005.0,23.0,NaN,NaN,10003.0,0.0
77,0,3800,AMETHYSTS,10002,1,9996.0,1.0,9995.0,29.0,10004,1,10005.0,29.0,NaN,NaN,10003.0,0.0
81,0,4000,AMETHYSTS,10002,9,9996.0,2.0,9995.0,27.0,10004,2,10005.0,27.0,NaN,NaN,10003.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19935,0,996700,AMETHYSTS,10002,7,9995.0,21.0,NaN,NaN,10005,21,NaN,NaN,NaN,NaN,10003.5,0.0
19952,0,997600,AMETHYSTS,10002,4,9995.0,24.0,NaN,NaN,10005,24,NaN,NaN,NaN,NaN,10003.5,0.0
19969,0,998400,AMETHYSTS,10002,4,9995.0,23.0,NaN,NaN,10005,23,NaN,NaN,NaN,NaN,10003.5,0.0
19977,0,998800,AMETHYSTS,10002,2,9995.0,26.0,NaN,NaN,10005,26,NaN,NaN,NaN,NaN,10003.5,0.0
